In [117]:
import pandas as pd
import numpy
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [118]:
jobs = pd.read_csv('dataset.csv')

In [119]:
# jobs.head(3)

In [120]:
jobs = jobs[['Job Title','Job Salary','Job Experience Required','Key Skills','Location','Industry','Role']]
# jobs.tail()

In [121]:
jobs.dropna(inplace=True)

In [122]:
jobs.isnull().sum()

Job Title                  0
Job Salary                 0
Job Experience Required    0
Key Skills                 0
Location                   0
Industry                   0
Role                       0
dtype: int64

In [123]:
jobs['Key Skills'] = jobs['Key Skills'].apply(lambda x: x.lower().replace("|"," ").replace("  "," ").strip())
jobs['Key Skills'].unique()[5]
jobs['Tags'] = jobs['Key Skills'] +  ' ' + jobs['Location'] + ' ' + jobs['Role']
jobs['Tags'] = jobs['Tags'].apply(lambda x: x.lower().replace("  "," ").strip())

In [124]:
cv = CountVectorizer(max_features=30000000,stop_words='english')

In [125]:
def vectorize(narr):
    vector = cv.fit_transform(narr)
    similarity = cosine_similarity(vector)
    return similarity

In [126]:
def getUserData():
    skills = input("Enter Your Skills (Mandatory) (ex: C, C++, Python)")
    experience = int(input("Enter Your Experience (ex: 0,1,2..)"))
    role = input("Enter If There are any Preffered Role (Optional)")
    location = input("Enter any Preffered Location (Optional)")
    userData = {'skills':skills, 'experience':experience, 'role':role, 'location':location }
    return userData

In [127]:
def recommendJobs():
    userData = getUserData()
    tags = userData['skills'] + ' ' + userData['location'] + ' ' + userData['role']
    temp = {
        'Job Title':' ',
        'Job Salary':" ",
        'Job Experience Required': userData['experience'] or ' ',
        'Key Skills': userData['skills'] or ' ',
        'Location':userData['location'] or ' ',
        'Industry':' ',
        'Role':userData['role'] or ' ',
        'Tags': tags.lower()
    }
    newDf = jobs.append(temp,ignore_index = True)
    tempArray = newDf['Tags'].values
    vector = vectorize(tempArray)
    recommended_list = sorted(list(enumerate(vector[-1])),reverse=True,key= lambda x:x[1])
    recList = []
    for i in recommended_list[1:10]:
        recList.append(jobs.iloc[i[0]])
    recommendedJobs = pd.DataFrame(recList)
    recommendedJobs = recommendedJobs[['Job Title','Job Salary','Job Experience Required', 'Key Skills','Location','Industry','Role']]
    return recommendedJobs

In [ ]:
recommendedJobs = recommendJobs()
recommendedJobs